# Labelfree 3D
As the downstream tasks for the image compression. From the brightfield image to the fluorescent image.

In [17]:
# import modules:
from pathlib import Path
import pandas as pd
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
import numpy as np
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter
from pathlib import Path
import os
import sys
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import monai
import torch
import random
from datetime import datetime
import shutil
from monai.transforms import (
    RandSpatialCropSamples,
    LoadImage,
    SaveImage,
    Compose,
    AddChannel,
    RepeatChannel,
    ToTensor,
    Transform,
    Transpose,
    CastToType,
    EnsureType,
    ScaleIntensityRangePercentiles,
    ScaleIntensity,
)
from tqdm.contrib import tenumerate

In [18]:
# define funcs and hyper-parameters:

_SEED = 2023
np.random.seed(_SEED)
random.seed(_SEED)
os.environ["PYTHONHASHSEED"] = str(_SEED)


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def normalizeItensity(image):
    # Convert the image data to a floating-point data type
    img = image.astype(np.float32)
    # Convert the image to a toch Tensor
    img = torch.from_numpy(img)
    # Normalize the intensity of the image using the MONAI NormalizeIntensity transform
    normalize_intensity = monai.transforms.NormalizeIntensity()
    img_normalized = normalize_intensity(img)
    # Convert the normalized image back to a numpy array
    return img_normalized.numpy()


def transform_img(image):
    img = image.astype(np.float32) / 65535
    return img


def compare_image(path1, path2, if_3d=True):
    # Load the two images
    if if_3d:
        image1 = AICSImage(path1).get_image_data("ZYX")
        image2 = AICSImage(path2).get_image_data("ZYX")
    else:
        image1 = AICSImage(path1).get_image_data("YX")
        image2 = AICSImage(path2).get_image_data("YX")
    # scale to 0-1
    scaler = ScaleIntensity()
    image1 = scaler(image1).cpu().numpy()
    image2 = scaler(image2).cpu().numpy()
    if if_3d:
        mse = np.sum((image1 - image2) ** 2) / (
            image1.shape[-1] * image1.shape[-2] * image1.shape[-3]
        )
    else:
        mse = np.sum((image1 - image2) ** 2) / (image1.shape[-1] * image1.shape[-2])

    ssim_value = ssim(image1, image2, data_range=1)
    psnr_value = psnr(image1, image2, data_range=1)
    corr = np.corrcoef(image1.ravel(), image2.ravel())[0, 1]
    # psnr = 10 * np.log10(1 / (mse + 0.000001))
    return mse, ssim_value, psnr_value, corr


def calculate_metric(fn_list1, fn_list2):
    mse_value = AverageMeter()
    ssim_value = AverageMeter()
    psnr_value = AverageMeter()
    corr_value = AverageMeter()
    assert len(fn_list1) == len(
        fn_list2
    ), f"should have the same length, but have {len(fn_list1)} and {len(fn_list2)}"

    for i, (fn1, fn2) in tenumerate(zip(sorted(fn_list1), sorted(fn_list2))):
        tmp_mse, tmp_ssim, tmp_psnr, tmp_corr = compare_image(fn1, fn2)
        mse_value.update(tmp_mse)
        ssim_value.update(tmp_ssim)
        psnr_value.update(tmp_psnr)
        corr_value.update(tmp_corr)
    print(
        "MSE:",
        mse_value.avg,
        "SSIM:",
        ssim_value.avg,
        "PSNR:",
        psnr_value.avg,
        "CORR:",
        corr_value.avg,
    )


class Normalize(Transform):
    def __init__(self):
        super().__init__()

    def __call__(self, img):
        # Rescale unint16 values to [0,1]
        result = img / 65535.0
        return result


def torch2img(x: torch.Tensor):
    # Convert  tensor to numpy array and rescale to uint16
    np_array = x.clamp_(0, 1).squeeze().cpu().detach().numpy()
    return np_array
    # return (np_array * (2**16 - 1)).astype(np.uint16)

In [ ]:
# run the run_im2im inference code for uncompressed data.
!run_im2im --config_path /mnt/eternus/users/Yu/project/data_compression/experiment/paper_exp/3d/labelfree/uncompressed_config.yaml

In [ ]:
# run the run_im2im inference code for compressed data.
!run_im2im --config_path /mnt/eternus/users/Yu/project/data_compression/experiment/paper_exp/3d/labelfree/compressed_config.yaml

# Evaluation

- gt and compressed
- gt and uncompressed
- compressed and uncompressed

In [14]:
# gt and compressed:
compress_dir = sorted(
    Path(
        "/mnt/eternus/users/Yu/project/data_compression/experiment/paper_exp/3d/labelfree/prediction/compressed"
    ).glob("*.tiff")
)
gt_dir = sorted(
    Path(
        "/mnt/eternus/users/Jianxu/projects/im2im_experiments_v1/data/labelfree3D/FBL/holdout"
    ).glob("*GT.tiff")
)
calculate_metric(compress_dir, gt_dir)

MSE: 0.0010345262097846528 SSIM: 0.8494615339342872 PSNR: 30.060456945449076 CORR: 0.5981160373078679


In [16]:
# gt and uncompressed:
uncompress_dir = sorted(
    Path(
        "/mnt/eternus/users/Yu/project/data_compression/experiment/paper_exp/3d/labelfree/prediction/uncompressed"
    ).glob("*.tiff")
)
gt_dir = sorted(
    Path(
        "/mnt/eternus/users/Jianxu/projects/im2im_experiments_v1/data/labelfree3D/FBL/holdout"
    ).glob("*GT.tiff")
)
calculate_metric(uncompress_dir, gt_dir)

MSE: 0.0005973280223174922 SSIM: 0.9268127516433369 PSNR: 32.60571312426753 CORR: 0.906572002216079


In [19]:
# compressed and uncompressed:
compress_dir = sorted(
    Path(
        "/mnt/eternus/users/Yu/project/data_compression/experiment/paper_exp/3d/labelfree/prediction/compressed"
    ).glob("*.tiff")
)
uncompress_dir = sorted(
    Path(
        "/mnt/eternus/users/Yu/project/data_compression/experiment/paper_exp/3d/labelfree/prediction/uncompressed"
    ).glob("*.tiff")
)
calculate_metric(compress_dir, uncompress_dir)

105it [07:41,  4.39s/it]

MSE: 0.0014884366131589371 SSIM: 0.8203372214022198 PSNR: 28.542511661908915 CORR: 0.6576589413189577
